## Install Libraries

In [0]:
%pip install -U -qqqq databricks-langchain langgraph langchain mlflow 
%restart_python

## Initialize Global Settings

In [0]:
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

## Test Prompt

In [0]:
llm.invoke("hello, world")

## REACT Agent

In [0]:
from langgraph.prebuilt import create_react_agent

def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

agent = create_react_agent(
    model=llm,  
    tools=[multiply],  
    prompt="You are a helpful assistant"  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is 1267 times 842"}]}
)